In [26]:
from collections import defaultdict
from rdkit import Chem
from IPython.display import display, Markdown

def composition(molecule, isotopes=False):
    if molecule:
        Chem.AddHs(molecule)
        comp = defaultdict(lambda: 0)
        for atom in molecule.GetAtoms():
            element_symbol = atom.GetSymbol()
            if not isotopes:
                comp[element_symbol] += 1
            else:
                isotope = atom.GetIsotope()
                try:
                    comp[element_symbol][isotope] += 1
                except:
                    comp[element_symbol] = defaultdict(lambda: 0)
                    comp[element_symbol][isotope] += 1
        return comp

def convert_smiles_to_markdown_formula(smiles, isotopes=False):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        comp = composition(mol, isotopes)

        isotopes = defaultdict(lambda: defaultdict(str))
        subscripts = defaultdict(lambda: defaultdict(int))
        superscripts = defaultdict(list)

        for element, counts in comp.items():
            for isotope, count in counts.items():
                if count > 1:
                    subscripts[element][isotope] = count
                if isotope != 0:
                    superscripts[element].append(isotope)

                isotopes[element][isotope] = 1
            superscripts[element].sort()

        formula = ""
        for element, isotope_count_pairs in isotopes.items():
            sorted_keys = sorted(isotope_count_pairs.keys())
            for isotope in sorted_keys:
                if element in superscripts:
                    if isotope in superscripts[element]:
                        formula += f"<sup>{isotope}</sup>"
                formula += element
                if element in subscripts:
                    if isotope in subscripts[element]:
                        for isotope, isotope_count in subscripts[element].items():
                            formula += f"<sub>{isotope_count}</sub>"
        display(Markdown(formula))
    else:
        return "Invalid SMILES"

# Example usage
smiles = "[13C]C[13C][18O]O[18O][16O][12C]"
convert_smiles_to_markdown_formula(smiles, isotopes=True)


C<sup>12</sup>C<sup>13</sup>C<sub>2</sub>O<sup>16</sup>O<sup>18</sup>O<sub>2</sub>